# Logic Optimizer
This assignment has been completed in group of 3. The group members are:<br>
Raghav Singhal - 19D070049  <br>
Sheel Shah - 19D070052<br>
Vipin Singh - 19D070069<br>

In [1]:
import numpy as np

In [2]:
def matrix_formation(function,num_literals):
    """Input the function as string in Sum of product forms and specify number of literals in the function"""
    #Can only be used if we have 26 literals and first literal starts from A
    #literal_alphabet = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
    # Capital Letter stands for false literal
    # Small Letter stands for true literal
    
    func_minterms = function.split("+")
    func_matrix = np.ones((len(func_minterms),num_literals*2),dtype = int)
    
    for minterm_num,minterm in enumerate(func_minterms):
        minterm = minterm.strip()
        minterm_literal = list(minterm)
        for i in minterm:
            if ord(i)>=97:
                true_literal = False
            else:
                true_literal = True
            literal_index = ord(i.upper()) - ord('A')
             
            if true_literal:
                func_matrix[minterm_num][2*literal_index] = 0
            else:
                func_matrix[minterm_num][2*literal_index+1] = 0
    
    return func_matrix

In [3]:
def cost_calculator(func_matrix,num_literals):
    """Calculate the cost of the function, input the circuit in matrix repersentation and specify number of literals"""
    total_cost = 0
    if func_matrix.size == 0:
        return 0
    for i in range(func_matrix.shape[0]):
        term_literal = 0
        for j in range(num_literals):
            if not (func_matrix[i][2*j] == 1 and func_matrix[i][2*j+1] == 1):
                term_literal +=1
            else:
                continue
        if term_literal != 1:
            total_cost += 2*term_literal + 2
    if func_matrix.shape[0] != 1:
        total_cost += 2*func_matrix.shape[0] +2
    
    return total_cost

In [4]:
def calc_cofactor(func_matrix,cofactor):
    """Calculate cofactor of the function w.r.t to the term. Input the function in matrix representation and cofactor is the term"""
    
    new_matrix = np.multiply(func_matrix,cofactor)
    void_implicant = []
    for i in range(new_matrix.shape[0]):
        for literal_num in range(func_matrix.shape[1]//2):
            if new_matrix[i][2*literal_num] == 0 and new_matrix[i][2*literal_num+1] == 0:
                void_implicant.append(i)
    new_func = np.delete(new_matrix,void_implicant,0)
    cofactor_complement = (cofactor + 1)%2
    final_function = (new_func + cofactor_complement)%2 
    return final_function              

In [5]:
def tautology_check(func_matrix):
    """Checks if the given function is a tautology or not. Input the fucntion in matrix representation"""
    if func_matrix.size == 0:
        return False
    matrix_shape = func_matrix.shape
    
    #column Sum
    columns_sum = np.sum(func_matrix,axis = 0)
    rows_sum = np.sum(func_matrix,axis = 1)
    for r_sum in rows_sum:
        if r_sum == matrix_shape[1]:
            return True
    
    num_not1_col = []
    next_literal_found = False
    for i,col_sum in enumerate(columns_sum):
        if col_sum == 0:
            return False
        elif col_sum != matrix_shape[0]:
            num_not1_col.append(i)
            if next_literal_found == False:
                next_literal_for_cofactor = i//2
                next_literal_found = True
                
    if len(num_not1_col) == 2 and num_not1_col[0]%2 ==0:
        if num_not1_col[1]-num_not1_col[0] == 1:
            return True
    
    cofactor = np.ones((1,func_matrix.shape[1]),dtype = int)
    
    #wrt to ture literal
    cofactor[0][2*next_literal_for_cofactor] = 0
    cofactor_func = calc_cofactor(func_matrix,cofactor)
    if not tautology_check(cofactor_func):
        return False
    #wrt to false literal
    
    cofactor[0][2*next_literal_for_cofactor] = 1
    cofactor[0][2*next_literal_for_cofactor+1] = 0
    cofactor_func = calc_cofactor(func_matrix,cofactor)
    if not tautology_check(cofactor_func):
        return False
    return True
    #Column Check

In [6]:
def containment_check(func_matrix,implicant):
    """Checks if the implicant is already covered by the function. Input the function in matrix representation and implicant."""
    if func_matrix.size == 0:
        return False
    cofactor_func = calc_cofactor(func_matrix,implicant)
    return tautology_check(cofactor_func)
    

In [7]:
def complement_calc(func_matrix,num_literals):
    """Calculate the complement function of the given funtion. Input the function in matrix repersentation and number of literals in the function"""
    if func_matrix.size == 0:
        return np.ones((1,2*num_literals),dtype = int)
    
    rows_sum = np.sum(func_matrix,axis = 1)
    for r_sum in rows_sum:
        if r_sum == func_matrix.shape[1]:
            return np.array([])
    
    if func_matrix.shape[0] == 1:
        #erro here
        new_matrix = np.array([])
        for i in range(func_matrix.shape[1]//2):
            temp = np.ones((1,func_matrix.shape[1]),dtype = int)
            #print(i)
            if func_matrix[0][2*i] == 1 and func_matrix[0][2*i+1] == 1:
                continue
            else:
                
                temp[0][2*i] = (func_matrix[0][2*i]+1)%2
                temp[0][2*i+1] = (func_matrix[0][2*i+1]+1)%2
                if new_matrix.size ==0:
                    new_matrix = temp
                else:
                    new_matrix = np.concatenate((new_matrix,temp))

        return new_matrix
    
    columns_sum = np.sum(func_matrix,axis = 0)
    col_check = True
    next_literal_found = False
    num_not1_col = []
    for i,col_sum in enumerate(columns_sum):
        if col_sum == 0:
            col_check = False
            if not next_literal_found:
                next_literal_found = True
                next_literal = i//2
            break
        elif col_sum != func_matrix.shape[0]:
            num_not1_col.append(i)
            if not next_literal_found:
                next_literal_found = True
                next_literal = i//2
                            
    if len(num_not1_col) == 2 and col_check and num_not1_col[0]%2 ==0:
        if num_not1_col[1]-num_not1_col[0] == 1:
            return np.array([])
    
    cofactor = np.ones((1,func_matrix.shape[1]),dtype = int)
    cofactor[0][2*next_literal] = 0
    cofactor_func_true = calc_cofactor(func_matrix,cofactor)
    #print("True Matrix \n",cofactor_func_true,"wrt:", next_literal)
    cofactor_complement_true =  complement_calc(cofactor_func_true,num_literals)
    #print("Wrt:", next_literal,"True \n",cofactor_complement_true)
    if cofactor_complement_true.size != 0:
        true_cofactor_comp = np.multiply(cofactor_complement_true,cofactor)
    
    cofactor[0][2*next_literal] = 1
    cofactor[0][2*next_literal+1] = 0
    
    cofactor_func_false = calc_cofactor(func_matrix,cofactor)
    #print(cofactor_func_false,"TP")
    cofactor_complement_false =  complement_calc(cofactor_func_false,num_literals)
    #print("False Matrix \n",cofactor_func_true,"wrt:", next_literal)
    #print("Wrt:", next_literal,"False \n",cofactor_complement_false)
    if cofactor_complement_false.size != 0:
        false_cofactor_comp = np.multiply(cofactor_complement_false,cofactor)
        if cofactor_complement_true.size != 0:
            return np.concatenate((false_cofactor_comp,true_cofactor_comp))
        else:
            return false_cofactor_comp
            
    elif cofactor_complement_true.size != 0:
        return true_cofactor_comp
    else:
        return np.array([])
    
    

In [8]:
def intersection_calculator(func_matrix,cube):
    """Returns the intersection of the function and the cube. Input the function and cube in matrix repersentation"""
    if func_matrix.size == 0 or cube.size == 0:
        return np.array([])
    inter_matrix = np.multiply(func_matrix,cube)
    
    del_row = []
    for i in range(func_matrix.shape[0]):
        for j in range(func_matrix.shape[1]//2):
            if inter_matrix[i][2*j]==0 and inter_matrix[i][2*j+1]==0:
                del_row.append(i)
    final_matrix = np.delete(inter_matrix,del_row,0)
    return final_matrix

In [9]:
def expand(onset_matrix,offset_matrix,num_literals):
    """Implement the expand. Input the onset, offset and number of literal of the funciton"""
    #row_heuristic
    columns_weights = np.sum(onset_matrix,axis = 0)
    weighted_onset = np.multiply(onset_matrix,columns_weights)
    weighted_sum = np.sum(weighted_onset,axis = 1)
    cube_order = np.argsort(weighted_sum)
    #Now we have the order to expand
    
    cube_included = np.zeros(onset_matrix.shape[0],dtype = int)
    new_matrix_cube = np.array([])


    for cube_index in cube_order:
        #print(cube_index)
        if cube_included[cube_index] == 1:
            continue
        
        cube = onset_matrix[cube_index][:]
        cube_included[cube_index] = 1
        for i in range(num_literals):
            new_cube = np.copy(cube)
            new_cube[2*i] = 1
            new_cube[2*i+1] = 1
            if intersection_calculator(offset_matrix,new_cube).size == 0:
                cube = np.copy(new_cube)

        if new_matrix_cube.size ==0:
            new_matrix_cube = cube.reshape((1,-1))
        else:
            new_matrix_cube = np.concatenate((new_matrix_cube,cube.reshape((1,-1))))
    
        for i in range(onset_matrix.shape[0]):
            if cube_included[i] == 1:
                continue
            else:
                if containment_check(np.copy(cube).reshape(1,-1),onset_matrix[i][:]):
                    cube_included[i] = 1
    return new_matrix_cube

In [10]:
def reduce(onset_matrix,dcare_matrix,num_literals):
    """Implement the reduce. Input the onset, offset and number of literal of the funciton"""
    #row_heuristic
    columns_weights = np.sum(onset_matrix,axis = 0)
    weighted_onset = np.multiply(onset_matrix,columns_weights)
    weighted_sum = np.sum(weighted_onset,axis = 1)
    cube_order = np.argsort(weighted_sum*-1)
    
    for cube_index in range(cube_order.size):
        cube = onset_matrix[cube_order[cube_index]][:]
        #Reduce Algo
        independent_literal = []
        for i in range(num_literals):
            if cube[2*i] == 1 and cube[2*i+1] == 1:
                independent_literal.append(i)
        for i in independent_literal:
            first_cube = np.copy(cube)
            first_cube[2*i] = 1
            first_cube[2*i+1] = 0
            
            second_cube= np.copy(cube)
            second_cube[2*i] = 0
            second_cube[2*i+1] = 1
            
            temp_cover = np.delete(onset_matrix,cube_index,0)
            
            #Second cube already Covered
            if containment_check(temp_cover,second_cube) or containment_check(dcare_matrix,second_cube):
                #print("First_cube",first_cube)
                cube = first_cube
            elif containment_check(temp_cover,first_cube) or containment_check(dcare_matrix,first_cube):
                cube = second_cube
                #print("Second cube",second_cube)
            else:
                #Cannot be reduce along this Literal
                continue
        onset_matrix[cube_order[cube_index]][:] = np.copy(cube)
        
    return onset_matrix

In [11]:
def irredundant(onset_matrix,dcare_matrix):
    """Implement the irredundant. Input the onset and offset of the funciton"""
    redundant_cube = []
    for i in range(onset_matrix.shape[0]):
        cube = onset_matrix[i][:]
        temp_cover = np.delete(onset_matrix,i,0)
        if containment_check(temp_cover,cube) or containment_check(dcare_matrix,cube):
            redundant_cube.append(i)
    if not redundant_cube:
        return onset_matrix

    return np.delete(onset_matrix,redundant_cube,0)

In [12]:
def essential(func_matrix):
    """Calculate the essential implicants of the function. Input the function in matrix representation"""
    not_essential = []
    essential = []
    for i in range(func_matrix.shape[0]):
        cube = func_matrix[i][:]
        temp_function = np.delete(func_matrix, i, 0)
        if containment_check(temp_function,cube):
            not_essential.append(i)
        else:
            essential.append(i)
    essential_matrix = np.delete(func_matrix, not_essential, 0)
    non_essential_matrix = np.delete(func_matrix, essential, 0)

    return essential_matrix,non_essential_matrix

In [13]:
def logic_optimizer(onset_matrix,num_literals,dcare_matrix = np.array([])):
    """Two level optimizater. Input the onset_matrix,num_literals and don't care matrix for the function"""
    if onset_matrix.size == 0:
        return 0
    elif dcare_matrix.size == 0:
        not_offset = onset_matrix
    else:
        not_offset = np.concatenate((onset_matrix,dcare_matrix))
        
    offset_matrix = complement_calc(not_offset,num_literals)
    current_optimized_matrix = np.copy(onset_matrix)
    
    expand_matrix_pre = expand(np.copy(onset_matrix),offset_matrix,num_literals)
    irredundant_matrix_pre = irredundant(expand_matrix_pre,dcare_matrix)
    
    essential_prime,func_minus_essential = essential(irredundant_matrix_pre)
    #print(essential_prime)
    if dcare_matrix.size == 0:
        dont_care = essential_prime
    elif essential_prime.size == 0:
        dont_care = dcare_matrix
    else:
        dont_care = np.concatenate((essential_prime,dcare_matrix))
    
    #essential calculator
    
    #Udating dont care
    current_optimized_matrix = np.copy(func_minus_essential)
    previous_cost = cost_calculator(current_optimized_matrix,num_literals)

    cost_tracker = []
    cost_tracker.append(previous_cost)
    
    while(True):
        #print(cost_tracker)
        #print(current_optimized_matrix,"Before")    
        reduced_matrix = reduce(np.copy(current_optimized_matrix),dont_care,num_literals)
        #print(current_optimized_matrix,"After")
        #print(reduced_matrix,"Before")
        expand_matrix = expand(reduced_matrix,offset_matrix,num_literals)
        #print(reduced_matrix,"After")
        #print(expand_matrix,"before")
        irredundant_matrix = irredundant(expand_matrix,dont_care)
        #print(expand_matrix,"After")
        current_cost = cost_calculator(irredundant_matrix,num_literals)
        cost_tracker.append(current_cost)
        
        if current_cost >= previous_cost:
            break
        elif current_cost == 0:
            current_optimized_matrix = np.copy(irredundant_matrix)
            break
        else:

            previous_cost = current_cost
            current_optimized_matrix = np.copy(irredundant_matrix)
        
    return np.concatenate((current_optimized_matrix,essential_prime))


## Logic Optimization for function with no Don't Care Cube
For implementing the logic optimization for function without a Don't Care cube, we can input the function as a string in the format specified below to the function optimize along with numer of literal in the function.
### String Format
The function can have upto 26 letters represented by english letter. A capital english letter is used for true literal and small letter is used for false literal.<br> For example <br>
01 = "aB"<br>
00 = "ab"<br>
10 = "Ab"<br>
11 = "AB"<br>
For input as sum of product, use the above rule and put a "+" sign in between products. <br>
For example <br>
00 + 11 can be represented as "ab + AB".<br>
Note: The spacing need not be considered any valid spacing is allowed.<br>

In [14]:
def optimize(function,num_literals,dont_care = np.array([])):
    """Optimize a function, Input takes two string, one for onset other for don't care set can be in form of string with capital letter impying true literal and small leter impying false literal"""
    matrix_form_func = matrix_formation(function,num_literals)
    if type(dont_care) == str:
        matrix_form_dont_care = matrix_formation(dont_care,num_literals)
    else:
        matrix_form_dont_care = dont_care
    return logic_optimizer(matrix_form_func,num_literals,matrix_form_dont_care)

In [15]:
#Example 1
f1 = "AB + Cd + CD +a"

print(optimize(f1,4))

[[1 1 0 1 1 1 1 1]
 [1 1 1 1 0 1 1 1]
 [1 0 1 1 1 1 1 1]]


In [16]:
#Example 2
f2 = "AB + Cd + CD + a"
print(optimize(f2,4))

[[1 1 0 1 1 1 1 1]
 [1 1 1 1 0 1 1 1]
 [1 0 1 1 1 1 1 1]]


In [17]:
#Example 3
f3 = "A + B + ab"
print(optimize(f3,2))

[[1 1 1 1]]


In [18]:
#Example 4
f4 = "ABC + ABc + C"
print(optimize(f4,3))

[[0 1 0 1 1 1]
 [1 1 1 1 0 1]]


In [19]:
#Example 5
f5 = "A +B"
fd5 = "ab"
print(optimize(f5,2,fd5))

[[1 1 1 1]]


In [20]:
#Example 6
f6 = "abc + Abc +aBc + abC"
fd6 = "ABc"
print(optimize(f6,3,fd6))

[[1 1 1 1 1 0]
 [1 0 1 0 1 1]]


In [22]:
#Example 8
f7 = "abcde + abCdE + abCDe + abCDE + aBCDe + aBCDE + Abcde +ABCDE"
fd7 = "aBcde + ABCDe + ABCdE + ABCde"
o7 = optimize(f7,5,fd7)
print(irredundant(o7,np.array([])))

[[1 1 1 0 1 0 1 0 1 0]
 [1 1 0 1 0 1 0 1 1 1]
 [1 0 1 0 0 1 1 1 0 1]
 [1 0 1 1 0 1 0 1 1 1]]
